# Plotting Figure 6: Graphical seasonal pattens of observed and predicted ET at 6 sites

In [1]:
# change date lab(month) to english from chinese
library(ggplot2)
library(scales)

Warning message:
"package 'ggplot2' was built under R version 3.5.3"

In [2]:
# x: the vector
# n: the number of samples
# centered: if FALSE, then average current sample and previous (n-1) samples
#           if TRUE, then average symmetrically in past and future. (If n is even, use one more sample from future.)
movingAverage <- function(x, n=1, centered=FALSE) {
    
    if (centered) {
        before <- floor  ((n-1)/2)
        after  <- ceiling((n-1)/2)
    } else {
        before <- n-1
        after  <- 0
    }

    # Track the sum and count of number of non-NA items
    s     <- rep(0, length(x))
    count <- rep(0, length(x))
    
    # Add the centered data 
    new <- x
    # Add to count list wherever there isn't a 
    count <- count + !is.na(new)
    # Now replace NA_s with 0_s and add to total
    new[is.na(new)] <- 0
    s <- s + new
    
    # Add the data from before
    i <- 1
    while (i <= before) {
        # This is the vector with offset values to add
        new   <- c(rep(NA, i), x[1:(length(x)-i)])

        count <- count + !is.na(new)
        new[is.na(new)] <- 0
        s <- s + new
        
        i <- i+1
    }

    # Add the data from after
    i <- 1
    while (i <= after) {
        # This is the vector with offset values to add
        new   <- c(x[(i+1):length(x)], rep(NA, i))
       
        count <- count + !is.na(new)
        new[is.na(new)] <- 0
        s <- s + new
        
        i <- i+1
    }
    
    # return sum divided by count
    s/count
}

## load and merge simulation results

In [3]:
workdir = 'D:/Field/NATT/modeling_et_afm_v16d/' # this maybe need change according where you place the directory of modeling_et

In [4]:
# Load data
fdata1 = read.csv(paste0(workdir, 'output/simulation_results.csv'))

fdata2 = read.csv(paste0(workdir, 'output/simulation_results_DIs.csv'))

In [5]:
# combine predicted et from the three models
edata = data.frame(site = fdata1$site,
                   date = as.Date(fdata1$date),
                   fe_qcflag = fdata1$fe_qcflag,   # latent heat flux quality flag
                   ebr       = fdata1$ebr,         # energy balance ratio
                   et_ob     = fdata1$et_ob,   
                   et_pml    = fdata1$et_pml,
                   et_sw     = fdata1$et_sw,
                   et_ts     = fdata1$et_ts,
                   et_pml_di = fdata2$et_pml_di,
                   et_sw_di  = fdata2$et_sw_di,
                   et_ts_di  = fdata2$et_ts_di)

In [6]:
# data quality control
# remove stp years 2009 and 2015 when soil water content observation is abnormal.
edata$year = as.numeric(strftime(as.Date(edata$date), format='%Y'))
edata = edata[edata$site != 'stp' | edata$year !=2009,]
edata = edata[edata$site != 'stp' | edata$year !=2015,]

In [7]:
# remove gap-filling percent >0.2
datas = subset(edata, fe_qcflag<0.2 & et_ob >0 & ebr > 0.7 & ebr < 1.3)

In [8]:
datas = na.omit(datas)

## calculate mean daily residuals in a whole year

In [9]:
datas$date = as.Date(datas$date)
datas$doy  = as.numeric(strftime(datas$date, format = "%j"))

In [10]:
# aggregate to doy for ET of each site
v_names = c('doy', 'et_ob', 'et_pml','et_sw', 'et_ts', 'et_pml_di', 'et_sw_di', 'et_ts_di')
lens = length(v_names) 
tmp = subset(datas, site == 'asm', select= v_names)
res_asm = aggregate(tmp[2: lens], list(tmp$doy), mean)
tmp = subset(datas, site == 'dap', select= v_names)
res_dap = aggregate(tmp[2: lens], list(tmp$doy), mean)
tmp = subset(datas, site == 'das', select= v_names)
res_das = aggregate(tmp[2: lens], list(tmp$doy), mean)
tmp = subset(datas, site == 'dry', select= v_names)
res_dry = aggregate(tmp[2: lens], list(tmp$doy), mean)
tmp = subset(datas, site == 'how', select= v_names)
res_how = aggregate(tmp[2: lens], list(tmp$doy), mean)
tmp = subset(datas, site == 'stp', select= v_names)
res_stp = aggregate(tmp[2: lens], list(tmp$doy), mean)

# residuals of 5 sites
res_asm$site = 'asm'; res_dap$site = 'dap'; res_das$site = 'das'; 
res_dry$site = 'dry'; res_how$site = 'how'; res_stp$site = 'stp'
res_all = rbind(res_asm, res_dap, res_das, res_dry, res_how, res_stp)

In [11]:
colnames(res_all) =  c(v_names, 'site')
head(res_all);tail(res_all)

doy,et_ob,et_pml,et_sw,et_ts,et_pml_di,et_sw_di,et_ts_di,site
1,1.5247523,1.8086399,1.2508136,1.2592552,1.6840863,1.0883021,1.191346,asm
2,1.9170939,2.2184054,1.5619178,1.5741766,1.9854180,1.2661933,1.395729,asm
3,1.4190002,1.7272599,1.4812460,1.4909778,1.5507538,1.2547033,1.364396,asm
4,1.4417092,2.1890870,2.1749283,2.1857979,2.0793531,2.0220611,2.139551,asm
5,0.3113496,0.5734018,0.5751254,0.5784379,0.3642785,0.3149107,0.312725,asm
6,1.5577685,1.5870193,1.6829967,1.6956522,1.4351363,1.4827690,1.603793,asm


,doy,et_ob,et_pml,et_sw,et_ts,et_pml_di,et_sw_di,et_ts_di,site
2188,360,1.948713,2.306744,2.558523,2.558523,2.143436,2.561394,2.561102,stp
2189,361,2.442051,2.391791,2.495499,2.495499,2.195141,2.494511,2.494229,stp
2190,362,2.527676,2.418623,2.668248,2.668248,2.201461,2.683490,2.683154,stp
2191,363,2.544142,2.281703,2.839338,2.839338,2.126623,2.857274,2.856984,stp
2192,364,2.449819,2.463237,2.812657,2.812657,2.294924,2.833193,2.832881,stp
2193,365,2.207245,2.372645,2.896925,2.896925,2.227185,2.917882,2.917568,stp


## Group data for ggplot

In [12]:
# group and stack data  according to modeltype
gdata_pml = cbind(subset(res_all, select= c('site', 'doy', 'et_ob', 'et_pml', 'et_pml_di')),
                  data.frame(model_type = rep('PML', dim(res_all)[1])))
gdata_sw = cbind(subset(res_all, select= c('site', 'doy', 'et_ob', 'et_sw', 'et_sw_di')),
                  data.frame(model_type = rep('SW', dim(res_all)[1])))
gdata_ts = cbind(subset(res_all, select= c('site', 'doy', 'et_ob', 'et_ts', 'et_ts_di')),
                  data.frame(model_type = rep('TS', dim(res_all)[1])))
colnames(gdata_pml) <- colnames(gdata_sw) <- colnames(gdata_ts) <- c('site', 'doy', 'obs', 'pre', 'pre_di', 'modeltype')
gdata = rbind(gdata_pml, gdata_sw, gdata_ts)

In [13]:
head(gdata); tail(gdata)

site,doy,obs,pre,pre_di,modeltype
asm,1,1.5247523,1.8086399,1.6840863,PML
asm,2,1.9170939,2.2184054,1.9854180,PML
asm,3,1.4190002,1.7272599,1.5507538,PML
asm,4,1.4417092,2.1890870,2.0793531,PML
asm,5,0.3113496,0.5734018,0.3642785,PML
asm,6,1.5577685,1.5870193,1.4351363,PML


,site,doy,obs,pre,pre_di,modeltype
6574,stp,360,1.948713,2.558523,2.561102,TS
6575,stp,361,2.442051,2.495499,2.494229,TS
6576,stp,362,2.527676,2.668248,2.683154,TS
6577,stp,363,2.544142,2.839338,2.856984,TS
6578,stp,364,2.449819,2.812657,2.832881,TS
6579,stp,365,2.207245,2.896925,2.917568,TS


In [14]:
gdata_obs = cbind(subset(gdata, select= c('site', 'doy', 'obs', 'modeltype')),
                  data.frame(val_type = rep('Observed', dim(gdata)[1])))
gdata_pre = cbind(subset(gdata, select= c('site', 'doy', 'pre', 'modeltype')),
                  data.frame(val_type = rep('Predicted without NEDI', dim(gdata)[1])))
gdata_pre_di = cbind(subset(gdata, select= c('site', 'doy', 'pre_di', 'modeltype')),
                  data.frame(val_type = rep('Predicted with NEDI', dim(gdata)[1])))
colnames(gdata_obs) <- colnames(gdata_pre) <- colnames(gdata_pre_di) <- c('site', 'doy', 'value', 'modeltype', 'valuetype')
gdatas = rbind(gdata_obs, gdata_pre, gdata_pre_di)

In [15]:
head(gdatas); tail(gdatas)

site,doy,value,modeltype,valuetype
asm,1,1.5247523,PML,Observed
asm,2,1.9170939,PML,Observed
asm,3,1.4190002,PML,Observed
asm,4,1.4417092,PML,Observed
asm,5,0.3113496,PML,Observed
asm,6,1.5577685,PML,Observed


,site,doy,value,modeltype,valuetype
19732,stp,360,2.561102,TS,Predicted with NEDI
19733,stp,361,2.494229,TS,Predicted with NEDI
19734,stp,362,2.683154,TS,Predicted with NEDI
19735,stp,363,2.856984,TS,Predicted with NEDI
19736,stp,364,2.832881,TS,Predicted with NEDI
19737,stp,365,2.917568,TS,Predicted with NEDI


In [16]:
gdatas$date = as.Date(gdatas$doy, origin = "2010-12-31")
gdatas$date = as.POSIXct(gdatas$date)
gdatas = gdatas[gdatas$doy<=365,]

In [17]:
pdata = gdatas
# Calculate ET 7-day moving average 
pdata$et_ma = NA
sites = c('asm', 'dap', 'das', 'dry', 'how', 'stp')
modeltypes = c('PML', 'SW', 'TS')
valuetypes = c('Observed', 'Predicted without NEDI', 'Predicted with NEDI')
for (i in sites)
    {
     for ( j in modeltypes)
         {
          for ( k in valuetypes)
             {
              pdata[pdata$site == i & pdata$modeltype == j & pdata$valuetype == k,]$et_ma = 
                 movingAverage(pdata[pdata$site == i & pdata$modeltype == j & pdata$valuetype == k, ]$value, 7, TRUE)
              }
     }
}
                                                      

In [18]:
summary(pdata)

     site                doy          value         modeltype 
 Length:19692       Min.   :  1   Min.   :0.05955   PML:6564  
 Class :character   1st Qu.: 92   1st Qu.:0.88834   SW :6564  
 Mode  :character   Median :183   Median :1.85495   TS :6564  
                    Mean   :183   Mean   :2.09097             
                    3rd Qu.:274   3rd Qu.:3.26349             
                    Max.   :365   Max.   :5.97686             
                  valuetype         date                         et_ma        
 Observed              :6564   Min.   :2011-01-01 08:00:00   Min.   :0.09493  
 Predicted without NEDI:6564   1st Qu.:2011-04-02 08:00:00   1st Qu.:0.89942  
 Predicted with NEDI   :6564   Median :2011-07-02 08:00:00   Median :1.85940  
                               Mean   :2011-07-02 08:36:51   Mean   :2.09090  
                               3rd Qu.:2011-10-01 08:00:00   3rd Qu.:3.27775  
                               Max.   :2011-12-31 08:00:00   Max.   :5.15992  

pdata[pdata$site == 'dry' & pdata$valuetype == 'observed',]$doy
###### doy 27 is missing at ade

In [19]:
new_df =data.frame(site = rep('dry',9), doy = rep(27,9), value = rep(NA,9), modeltype = rep(c('PML','SW', 'TS'),3), 
                   valuetype = c(rep('Observed', 3), rep('Predicted without NEDI', 3), rep('Predicted with NEDI', 3)), 
                   date = rep(as.Date('2011-01-27'), 9), et_ma = rep(NA,9))
pdata = rbind(pdata, new_df)
summary(pdata)

     site                doy          value         modeltype 
 Length:19701       Min.   :  1   Min.   :0.05955   PML:6567  
 Class :character   1st Qu.: 92   1st Qu.:0.88834   SW :6567  
 Mode  :character   Median :183   Median :1.85495   TS :6567  
                    Mean   :183   Mean   :2.09097             
                    3rd Qu.:274   3rd Qu.:3.26349             
                    Max.   :365   Max.   :5.97686             
                                  NA's   :9                   
                  valuetype         date                         et_ma        
 Observed              :6567   Min.   :2011-01-01 08:00:00   Min.   :0.09493  
 Predicted without NEDI:6567   1st Qu.:2011-04-02 08:00:00   1st Qu.:0.89942  
 Predicted with NEDI   :6567   Median :2011-07-02 08:00:00   Median :1.85940  
                               Mean   :2011-07-02 06:54:12   Mean   :2.09090  
                               3rd Qu.:2011-10-01 08:00:00   3rd Qu.:3.27775  
                      

In [20]:
# set facet order
pdata$siten = NA
pdata[pdata$site=='how', ]$siten = 'AU-How'
pdata[pdata$site=='dap', ]$siten = 'AU-DaP'
pdata[pdata$site=='das', ]$siten = 'AU-DaS'
pdata[pdata$site=='dry', ]$siten = 'AU-Dry'
pdata[pdata$site=='stp', ]$siten = 'AU-StP'
pdata[pdata$site=='asm', ]$siten = 'AU-ASM'
pdata$siten <- factor(pdata$siten, level=c('AU-How','AU-DaS','AU-Dry', 'AU-ASM', 'AU-DaP','AU-StP'),
                    labels=c('AU-How','AU-DaS','AU-Dry', 'AU-ASM', 'AU-DaP','AU-StP'))
pdata$modeltype <- factor(pdata$modeltype, level=c('SW', 'PML', 'TS'), labels=c('SW', 'PML', 'TS'))

In [21]:
# remove data in AU-DaP and AU-Stp for TS model since there are not three layers
pdata[pdata$site == 'dap' & pdata$modeltype == 'TS', ]$et_ma = NA
pdata[pdata$site == 'stp' & pdata$modeltype == 'TS', ]$et_ma = NA

### Function plot moving average

In [22]:
pdata = na.omit(pdata)

In [23]:
col_model = c('black', 'green3', 'red')

In [24]:
Sys.setlocale("LC_TIME", "English") # Windows

[1] "English_United States.1252"

In [25]:
#options(repr.plot.with=10, repr.plot.height =9)
ts_scatter <- ggplot(aes(x = date), data = pdata)+  
  #geom_hline(yintercept=0, lty=1, colour = gray(0.8)) +
    facet_grid(siten ~ modeltype) + 
  geom_line(aes(y = et_ma, colour= factor(valuetype)), size=0.2) +
  scale_y_continuous(
    name = expression(paste('ET observed vs. predicted  (mm d'^'-1',')')), 
    limits = c(0, 6)) +
  theme_bw()+
  theme(panel.grid.major = element_blank(),
    panel.grid.minor = element_blank()) +
  scale_color_manual(values=col_model)  

ts_scatter2 = ts_scatter +
  theme(legend.position = "bottom") +
  scale_x_datetime(labels = date_format("%b"),
                 breaks = seq(as.POSIXct("2011-01-01 08:00:00"),
                 as.POSIXct("2011-12-31 08:00:00"), "1 months")) +
  xlab('Date')+
  theme(axis.text.x = element_text(angle=70, hjust=0.8)) +
  labs(color="") 

#ts_scatter2

In [27]:
model_name ='6sites'
postscript(paste0('Fig.6_et_obs_pre_', model_name,'.eps'),width=6, height=6)
print(ts_scatter2)
dev.off()


png 
  2

## Plot seasonal preciptation

In [27]:
month12 = seq(as.POSIXct("2011-01-01 08:00:00"),
                 as.POSIXct("2011-12-31 08:00:00"), "1 months")

In [28]:
# assign monthly precipitation to daily values
mrain = read.csv(file = paste0(workdir,'data/monthly_precipitation_6sites.csv'))

In [29]:
mrain$date = rep(month12, 6)

In [30]:
head(mrain)

X,month,site,precip,date
1,1,asm,90.828571,2011-01-01 08:00:00
2,2,asm,45.057143,2011-02-01 08:00:00
3,3,asm,28.428571,2011-03-01 08:00:00
4,4,asm,15.085714,2011-04-01 08:00:00
5,5,asm,12.314286,2011-05-01 08:00:00
6,6,asm,4.285714,2011-06-01 08:00:00


In [31]:
pdata = mrain
# set facet order
pdata$siten = NA
pdata[pdata$site=='how', ]$siten = 'AU-How'
pdata[pdata$site=='dap', ]$siten = 'AU-DaP'
pdata[pdata$site=='das', ]$siten = 'AU-DaS'
pdata[pdata$site=='dry', ]$siten = 'AU-Dry'
pdata[pdata$site=='stp', ]$siten = 'AU-StP'
pdata[pdata$site=='asm', ]$siten = 'AU-ASM'
pdata$siten <- factor(pdata$siten, level=c('AU-How','AU-DaS','AU-Dry', 'AU-ASM', 'AU-DaP','AU-StP'),
                    labels=c('AU-How','AU-DaS','AU-Dry', 'AU-ASM', 'AU-DaP','AU-StP'))

In [32]:
#options(repr.plot.with=10, repr.plot.height =9)
ts_scatter <- ggplot(aes(x = date), data = pdata)+  
  #geom_hline(yintercept=0, lty=1, colour = gray(0.8)) +
    facet_wrap(~siten, ncol=3) + 
  geom_bar(mapping = aes(x = date, y = precip ), stat = "identity", colour = gray(0.4), fill = gray(0.4))+ 
  scale_y_continuous(
    name = 'Monthly precipitation (mm)', 
    #sec.axis = sec_axis(~ . * 300 , name = "Monthly  precipitation (mm)"), 
    limits = c(0, 500)) +
  theme_bw()+
  theme(panel.grid.major = element_blank(),
    panel.grid.minor = element_blank()) +
  scale_color_manual(values=col_model)  

ts_scatter2 = ts_scatter +
  theme(legend.position = "bottom") +
  scale_x_datetime(labels = date_format("%b"),
                 breaks = seq(as.POSIXct("2011-01-01 08:00:00"),
                 as.POSIXct("2011-12-31 08:00:00"), "1 months")) +
  xlab('Date')+
  theme(axis.text.x = element_text(angle=70, hjust=0.8)) +
  labs(color="") 

#ts_scatter2

In [33]:
model_name ='6sites'
postscript(paste0('Fig.S4_monthly_precip_', model_name,'2.eps'),width=8, height=5)
print(ts_scatter2)
dev.off()


png 
  2